### Import

In [3]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ----- ---------------------------------- 16.8/124.9 MB 86.2 MB/s eta 0:00:02
   ------------- ------------------------- 41.9/124.9 MB 102.8 MB/s eta 0:00:01
   --------------------- ----------------- 67.4/124.9 MB 108.1 MB/s eta 0:00:01
   ---------------------------- ---------- 92.3/124.9 MB 110.4 MB/s eta 0:00:01
   ----------------------------------- -- 117.2/124.9 MB 111.6 MB/s eta 0:00:01
   -------------------------------------- 124.9/124.9 MB 101.7 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder

import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

### Data Load

In [2]:
train = pd.read_csv('./Data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./Data/test.csv').drop(columns=['ID'])

In [3]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [4]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [5]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [6]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [7]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [8]:
numeric_columns = [col for col in numeric_columns if col in X.columns]

X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_encoded, y, test_size=0.2, random_state=42, stratify=y
)

### Train

XGBoost

In [13]:
def objective(trial):
    age_weight = trial.suggest_float("age_weight", 1.0, 3.0, step=0.1)

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "lambda": trial.suggest_float("lambda", 1, 10),
        "alpha": trial.suggest_float("alpha", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "feature_weights": [age_weight if col == '시술 당시 나이' else 1 for col in X_train.columns]
    }

    model = xgb.XGBClassifier(
        **params,
        objective="binary:logistic",
        eval_metric="auc",
        use_label_encoder=False,
        verbosity=0,  # 경고 메시지 숨김
        random_state=42,
        n_jobs=-1,
        scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1])
    )

    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    y_pred = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)

    return auc


In [14]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, show_progress_bar=True)

[I 2025-02-22 23:35:03,048] A new study created in memory with name: no-name-cd97fcb0-ab59-434d-a628-06a34eb65101


  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\vhehr\AppData\Local\Temp\ipykernel_28848\1594092381.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),


[W 2025-02-22 23:36:02,062] Trial 0 failed with parameters: {'age_weight': 2.0, 'n_estimators': 700, 'max_depth': 4, 'learning_rate': 0.017279093008227418, 'subsample': 0.8682947558813573, 'colsample_bytree': 0.7700965300965049, 'gamma': 3.1898355216430376, 'lambda': 9.70928652398454, 'alpha': 9.010579922608988, 'min_child_weight': 8} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\vhehr\.conda\envs\LGAimers6\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\vhehr\AppData\Local\Temp\ipykernel_28848\1594092381.py", line 28, in objective
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vhehr\.conda\envs\LGAimers6\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\vhehr\.conda\envs\LGAimers6\L

KeyboardInterrupt: 

In [26]:
import joblib

# Optuna 최적화 실행 후 최적의 study 저장
joblib.dump(study, "/xgb_optuna_study2.pkl")


['/xgb_optuna_study2.pkl']

In [12]:
# 최적 하이퍼파라미터 가져오기
best_params = study.best_params

# 최적의 파라미터로 모델 재학습
model = xgb.XGBClassifier(
    **best_params,
    objective="binary:logistic",
    eval_metric="auc",
    use_label_encoder=False,
    verbosity=0,
    random_state=42,
    n_jobs=-1
)

In [13]:
model.fit(X_train_encoded, y)

XGBClassifier(alpha=8.94379029687546, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9798171718247131, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=4.568361223056192,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=3.2578449896074764,
              learning_rate=0.011084576424821734, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=9, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=850, n_jobs=-1, ...)

In [20]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 예측
y_train_pred = model.predict(X_train_encoded)
y_train_proba = model.predict_proba(X_train_encoded)[:, 1]  # ROC-AUC Score 계산용

# 평가
accuracy = accuracy_score(y, y_train_pred)
roc_auc = roc_auc_score(y, y_train_proba)

# 출력
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

Accuracy: 0.7480
ROC-AUC Score: 0.7444


### Predict

In [21]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [22]:
# sample_submission = pd.read_csv('./Data/sample_submission.csv')
sample_submission = pd.read_csv('E:/인공지능/LG Aimers 6기/온라인 해커톤/code/Data/sample_submission.csv')

sample_submission['probability'] = pred_proba

In [23]:
# sample_submission.to_csv('./XGBoost_SMOTE3_optimization_submit.csv', index=False)
sample_submission.to_csv('E:/인공지능/LG Aimers 6기/온라인 해커톤/code/XGBoost_Weighted (나이 x 1.5)_optimization_submit.csv', index=False)